In [1]:
import numpy as np
import mlflow
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
from imblearn.combine import SMOTETomek
warnings.filterwarnings('ignore')


In [2]:
X,y = make_classification(n_samples=1000,n_features=10,n_informative=3,n_redundant=7,
                          weights=[0.9,0.1],flip_y=0,random_state=23)

In [3]:
np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=3)

In [5]:
params = {
    "solver":"lbfgs",
    "max_iter":1000,
    "multi_class":"auto",
    "random_state":8888,
}

lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=8888)

In [6]:
y_pred = lr.predict(X_test)
report = classification_report(y_test,y_pred)

In [7]:
print(report)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       270
           1       0.93      0.83      0.88        30

    accuracy                           0.98       300
   macro avg       0.95      0.91      0.93       300
weighted avg       0.98      0.98      0.98       300



In [8]:
report_dict= classification_report(y_test,y_pred,output_dict=True)

In [9]:
report_dict

{'0': {'precision': 0.9816849816849816,
  'recall': 0.9925925925925926,
  'f1-score': 0.9871086556169428,
  'support': 270},
 '1': {'precision': 0.9259259259259259,
  'recall': 0.8333333333333334,
  'f1-score': 0.8771929824561403,
  'support': 30},
 'accuracy': 0.9766666666666667,
 'macro avg': {'precision': 0.9538054538054538,
  'recall': 0.912962962962963,
  'f1-score': 0.9321508190365415,
  'support': 300},
 'weighted avg': {'precision': 0.976109076109076,
  'recall': 0.9766666666666667,
  'f1-score': 0.9761170883008626,
  'support': 300}}

In [10]:
import mlflow

In [11]:
mlflow.set_experiment("First Experiment")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5001")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy':report_dict['accuracy'],
        'recall_class_0':report_dict['0']['recall'],
        'recall_class_1':report_dict['1']['recall'],
        'f1_score_macro':report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr,'logistic regression')

In [12]:
import numpy as np
import sklearn
from imblearn.combine import SMOTETomek

print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Imbalanced-learn imported successfully.")


Numpy version: 1.24.0
Scikit-learn version: 1.1.3
Imbalanced-learn imported successfully.


In [13]:
from imblearn.combine import SMOTETomek

In [14]:
smt = SMOTETomek(random_state=34)

X_train_res,y_train_res = smt.fit_resample(X_train,y_train)

In [15]:
np.unique(y_train,return_counts=True)

(array([0, 1]), array([630,  70]))

In [16]:
np.unique(y_train_res,return_counts=True)

(array([0, 1]), array([627, 627]))

In [19]:
models = [
    (
        "LogisticRegression",
        {"C":1,"solver":"liblinear"},
        LogisticRegression(),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "Random_Forest",
        {"n_estimators":30,"max_depth":3},
        RandomForestClassifier(),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [38]:
logistic_models = [
    ("LogisticRegression_default",
 {"C": 1.0, "solver": "lbfgs"},
 LogisticRegression(),
 (X_train, y_train),
 (X_test, y_test)),
("LogisticRegression_L1",
 {"C": 0.1, "solver": "liblinear", "penalty": "l1"},
 LogisticRegression(C=0.1, solver="liblinear", penalty="l1"),
 (X_train, y_train),
 (X_test, y_test))
,
    ("LogisticRegression_L2",
 {"C": 0.5, "solver": "lbfgs", "penalty": "l2"},
 LogisticRegression(C=0.5, solver="lbfgs", penalty="l2"),
 (X_train, y_train),
 (X_test, y_test))
,
    ("LogisticRegression_no_reg",
 {"C": 1e6, "solver": "lbfgs"},
 LogisticRegression(C=1e6, solver="lbfgs"),
 (X_train, y_train),
 (X_test, y_test)),
 (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )

]

In [39]:
reports = []

for model_name,model_parms,model,train,test in logistic_models:
    X_train = train[0]
    X_test = test[0]
    y_train = train[1]
    y_test = test[1]
    model.set_params(**model_parms)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred,output_dict=True)
    reports.append(report)


In [22]:
reports

[{'0': {'precision': 0.9816176470588235,
   'recall': 0.9888888888888889,
   'f1-score': 0.9852398523985241,
   'support': 270},
  '1': {'precision': 0.8928571428571429,
   'recall': 0.8333333333333334,
   'f1-score': 0.8620689655172413,
   'support': 30},
  'accuracy': 0.9733333333333334,
  'macro avg': {'precision': 0.9372373949579832,
   'recall': 0.9111111111111112,
   'f1-score': 0.9236544089578826,
   'support': 300},
  'weighted avg': {'precision': 0.9727415966386553,
   'recall': 0.9733333333333334,
   'f1-score': 0.972922763710396,
   'support': 300}},
 {'0': {'precision': 0.9746376811594203,
   'recall': 0.9962962962962963,
   'f1-score': 0.9853479853479854,
   'support': 270},
  '1': {'precision': 0.9583333333333334,
   'recall': 0.7666666666666667,
   'f1-score': 0.8518518518518519,
   'support': 30},
  'accuracy': 0.9733333333333334,
  'macro avg': {'precision': 0.9664855072463768,
   'recall': 0.8814814814814815,
   'f1-score': 0.9185999185999186,
   'support': 300},
  'w

In [40]:
mlflow.set_experiment("LogisticExp")
mlflow.set_tracking_uri("http://127.0.0.1:5001")

for i,element in enumerate(logistic_models):
    model_name = element[0]
    model_parms = element[1]
    model_parms['model_name']=model_name
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(model_parms)
        mlflow.log_metrics(
            {
                'accuracy':report['accuracy'],
                'recall_class_1': report['1']['recall'],
                'recall_class_0':report['0']['recall'],
                'f1_score_macro': report['macro avg']['f1-score']
            }
        )

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model,"model")
        else:
            mlflow.sklearn.log_model(model,"model")

2024/11/18 22:06:34 INFO mlflow.tracking.fluent: Experiment with name 'LogisticExp' does not exist. Creating a new experiment.
